In [1]:
from helpers import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
from scipy.interpolate import griddata
from scipy.interpolate import interp1d
from scipy.stats import linregress
import matplotlib.colors as colors


In [24]:
def load_fenics_results(csv):
    df_filtered = pd.read_csv(f'{csv}')
    df_surface = df_filtered[df_filtered['d']<=1e-6]
    df = df_filtered
    new_rows = []
    
    # Loop through all unique z values
    for z_val in df['z'].unique():
        # Subset the dataframe for this z
        df_z = df[df['z'] == z_val]
    
        # Only interpolate if x spans 0
        if df_z['x'].min() <= 0 <= df_z['x'].max():
            # Sort by x to ensure correct interpolation
            df_z = df_z.sort_values('x')
    
            # Interpolate each variable at x=0
            new_row = {
                'x': 0,
                'z': z_val,
            }
            for col in ['u', 'w', 'rho', 'p', 'mu', 'd']:
                new_row[col] = np.interp(
                    0, df_z['x'], df_z[col]
                )
            new_rows.append(new_row)
    
    # Convert new rows to DataFrame
    df_interp = pd.DataFrame(new_rows)
    
    # Combine with original DataFrame
    df_combined = pd.concat([df, df_interp], ignore_index=True)
    df_divide = df_combined[df_combined['x']==0]
    df_bed = df_filtered[df_filtered['z']==0]
    left_x = df_filtered['x'].min()
    right_x = df_filtered['x'].max()
    df_flanks = df_filtered[(df_filtered['x'] == left_x) | (df_filtered['x'] == right_x)]
    # need to add a section that calculates a mu flank estimate, and reassigns df_flanks['mu'] accordingly
    return df_filtered, df_surface, df_divide, df_bed, df_flanks

In [25]:
df_filtered, df_surface, df_divide, df_bed, df_flanks = load_fenics_results('fenics_data.csv')

In [32]:
df_filtered[abs(df_filtered['x'])<2000]

,x,z,u,w,rho,p,mu,d
20,-1836.734694,-0.000000,0.000000e+00,-5.572603e-14,917.0,5.572738e+06,9.933706e+13,621.066411
21,-1428.571429,-0.000000,0.000000e+00,-5.590472e-14,917.0,5.590584e+06,9.944749e+13,622.936612
22,-1020.408163,-0.000000,0.000000e+00,-5.607072e-14,917.0,5.607184e+06,9.948838e+13,624.628567
23,-612.244898,-0.000000,0.000000e+00,-5.621713e-14,917.0,5.621840e+06,9.942260e+13,626.099708
24,-204.081633,-0.000000,0.000000e+00,-5.631068e-14,917.0,5.631216e+06,9.937164e+13,627.255355
...,...,...,...,...,...,...,...,...
7368,107.603420,64.784910,1.780365e-09,-5.858734e-10,917.0,5.046224e+06,1.560313e+14,562.669570
7370,1519.274377,62.141921,2.246999e-08,-4.905639e-10,917.0,5.024999e+06,1.531694e+14,560.393434
7371,1835.236871,62.087555,2.723432e-08,-5.209793e-10,917.0,5.011382e+06,1.530935e+14,558.986012
7372,-1501.375668,517.759972,-5.545718e-08,-1.549004e-08,917.0,6.615173e+05,3.565012e+15,104.855241
